In [2]:

from utils import *
from align_and_merge import *
import matplotlib.pyplot as plt
import time
from main import *
# from imported_fft_util import *
import numpy as np

%load_ext autoreload
%autoreload 2

In [3]:
# from rocket_fft import numpy_like
# numpy_like()

## General Align

In [4]:
image_name = "c1b1_20150226_144326_422"
raw_images = load_raw_images_parallel(image_name)
ref_image_index = select_reference_image(raw_images)

In [5]:
print("Reference image index:", ref_image_index)

Reference image index: 0


In [6]:
downsampled_raw_images = np.array([gaussian_downsample(raw_image, 2) for raw_image in raw_images])
print("downsampled raw images with shape: ", downsampled_raw_images[0].shape, "previous shape: ", raw_images[0].shape)

downsampled raw images with shape:  (1232, 1640) previous shape:  (2464, 3280)


In [7]:
# Generate alignment matrix using pyramid block matching
start = time.time()
reference_motion_matrix = burst_align(ref_image_index, downsampled_raw_images, parallel=True)
print("Time taken for parallel alignment: ", time.time() - start)

pyramid shapes:  [(19, 25), (77, 102), (308, 410), (1232, 1640)]
Time taken for parallel alignment:  11.873320579528809


In [8]:
motion_matrix = upsample_image(reference_motion_matrix, raw_images.shape[1], raw_images.shape[2]) * 2

start = time.time()
aligned_burst_patches = parallel_align(motion_matrix, raw_images)
print("Time taken for pixel shift alignment: ", time.time() - start)

(10, 2464, 3280, 3) (10, 2464, 3280)
Time taken for pixel shift alignment:  0.3752157688140869


## Naive Merge

In [9]:
start = time.time()
final_merged_frame = merge_images(aligned_burst_patches, ref_image_index)
print("Time taken for naive merge: ", time.time() - start)

100%|██████████| 154/154 [00:09<00:00, 16.91it/s]

Time taken for naive merge:  9.129734516143799


## Optimized Merge

In [10]:
@njit
def jit_fft(x):
    return np.fft.fft(x)

@njit
def jit_ifft(x):
    return np.fft.ifftn(x)

In [11]:
start = time.time()
aligned_burst_patches = np.array(aligned_burst_patches)
final_merged_frame = np.zeros_like(aligned_burst_patches[0])
final_merged_frame = parallel_merge_images(aligned_burst_patches, ref_image_index, final_merged_frame)
print("Time taken for parallel merge: ", time.time() - start)

Time taken for parallel merge:  4.068123817443848
